In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9fcbd02e60890ddc4c4848b9c7205515ebf6ab218cbeb23148f33b7ce863322b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [8]:
data=spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [9]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [10]:
data=data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [11]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import length

In [13]:
data=data.withColumn('length',length(data['text']))

In [14]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [15]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [16]:
from pyspark.ml.feature import (StringIndexer,VectorAssembler,Tokenizer,StopWordsRemover,CountVectorizer,
                                IDF)

In [18]:
tokenizer=Tokenizer(inputCol='text',outputCol='token_text')
stop_remove=StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec=CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf=IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric=StringIndexer(inputCol='class',outputCol='label')

In [22]:
clean_up=VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [55]:
from pyspark.ml.classification import NaiveBayes,RandomForestClassifier,DecisionTreeClassifier

In [54]:
nb=NaiveBayes()

In [57]:
rfc=RandomForestClassifier()
dtc=DecisionTreeClassifier()

In [25]:
from pyspark.ml import Pipeline

In [26]:
data_prep=Pipeline(stages=[ham_spam_to_numeric,tokenizer,
                           stop_remove,count_vec,idf,clean_up])

In [27]:
cleaner=data_prep.fit(data)

In [29]:
clean_data=cleaner.transform(data)

In [32]:
clean_data=clean_data.select('label','features')

In [33]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [34]:
(train_data,test_data)=clean_data.randomSplit([0.7,0.3])

In [35]:
spam_detector=nb.fit(train_data)

In [58]:
spam_detector_rfc=rfc.fit(train_data)
spam_detector_dtc=dtc.fit(train_data)

In [36]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [37]:
test_results=spam_detector.transform(test_data)

In [59]:
test_results_rfc=spam_detector_rfc.transform(test_data)
test_results_dtc=spam_detector_dtc.transform(test_data)

In [38]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-815.51459093778...|[1.0,1.0415899046...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-575.69807263924...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-886.48086427219...|[1.0,4.2250741481...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1191.8558295935...|[1.0,3.6202976198...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-757.92511378945...|[1.0,1.8568824775...|       0.0|
|  0.0|(13424,[0,1,27,85...|[-1523.3770915088...|[0.99999986970940...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1137.2377472112...|[3.64611018719485...|       1.0|
|  0.0|(13424,[0,1,150,1...|[-250.82077753372...|[0.98868192058953...|       0.0|
|  0.0|(13424,[0,1,428,6...|[-324.31698592898...|[0.99999924736040...|       0.0|
|  0.0|(13424,[0

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
acc_eval=MulticlassClassificationEvaluator()

In [51]:
acc=acc_eval.evaluate(test_results)

In [52]:
acc

0.9314825697091209

In [60]:
acc_rfc=acc_eval.evaluate(test_results_rfc)
acc_rfc

0.8178728745402478

In [61]:
acc_dtc=acc_eval.evaluate(test_results_dtc)
acc_dtc

0.9430139826529719